# Leave 2 Participants Out (L2PO)

The Leave-Two-Participants-Out (L2PO) cross-validation was applied, meaning the model was trained on all participants except two, and then tested on the two that were left-out. 

In [ ]:
# IMPORTS
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from scipy.stats import pearsonr
import statsmodels.api as sm
from lightgbm import LGBMRegressor
from typing import List
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [49]:
# Defining the directories
home_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion"
full_video_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Features-NEW/Full"
intervals_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Features-NEW/Intervals"
ground_truth_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth"
results_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Results"

**Loading the data** - will be using only Pupil and GSR now.

In [ ]:
# Full Pupil - loading the csv
df_pupil_full = pd.read_csv(os.path.join(full_video_directory, "Not_Interval_60_part_all_stat_features_12062025.csv"))
df_pupil_full.rename(columns={"Participant": "participant", "simuli_name_1": "video"}, inplace=True)
df_pupil_full.drop(columns=['Arousal', 'Valence', 'simuli_name_2','Unnamed: 0'], inplace=True)

# Intervals Pupil - Loading the csv
df_pupil_interval = pd.read_csv(os.path.join(intervals_directory, "Interval_60_part_all_stat_features_01062025.csv"))
df_pupil_interval.rename(columns={"Participant": "participant", "simuli_name_1": "video"}, inplace=True)
df_pupil_interval.drop(columns=['Arousal', 'Valence', 'simuli_name_2', 'Unnamed: 0'], inplace=True)

# Merging the intervals and full GSR features
df_pupil = pd.merge(df_pupil_full, df_pupil_interval, on=['participant', 'video'],
                    suffixes=('_whole', '_interval'))


# Full GSR - Loading the csv
df_gsr_full = pd.read_csv(os.path.join(full_video_directory, "gsr_features.csv"))
df_gsr_full.rename(columns={"ParticipantID": "participant", "StimulusName": "video"}, inplace=True)

# Intervals GSR - Loading the csv
df_gsr_intervals = pd.read_csv(os.path.join(intervals_directory, "gsr_features_intervals.csv"))
df_gsr_intervals.rename(columns={"ParticipantID": "participant", "StimulusName_0": "video"}, inplace=True)

# Merging the intervals and full GSR features
df_gsr = pd.merge(df_gsr_full, df_gsr_intervals, on=['participant', 'video'],
                  suffixes=('_whole', '_interval'))


participant_col = 'participant'
video_col = 'video'

pupil_features = [col for col in df_pupil.columns if col not in [participant_col, video_col]]
gsr_features = [col for col in df_gsr.columns if col not in [participant_col, video_col]]


# Merging df_fer and df_pupil
df = pd.merge( df_gsr,df_pupil, on=['participant', 'video'], how='inner')

In [ ]:
# Checking the number of participants
print('Participants Pupil:', len(df_pupil['participant'].unique()))
print('Participants GSR:', len(df_gsr['participant'].unique()))
print('Participants merged:', len(df['participant'].unique()))

Participants Pupil: 59
Participants GSR: 59
Participants merged: 56


Cleaning the data

In [ ]:
# Get columns with NaNs and sort by number of NaNs
nan_summary = df.isnull().sum()
nan_summary = nan_summary[nan_summary > 0].sort_values(ascending=False)

# Display the columns with NaNs
print(nan_summary.head(100))

# Column names with NaNs from nan_summary
missing_cols = nan_summary.index.tolist()

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')  
df[missing_cols] = imputer.fit_transform(df[missing_cols])

corr_kurtosis_whole       1619
corr_skewness_whole       1619
corr_auc_whole            1619
diff_kurtosis_whole       1619
diff_skewness_whole       1619
diff_auc_whole            1619
corr_kurtosis_interval     668
corr_skewness_interval     668
corr_auc_interval          668
diff_kurtosis_interval     668
diff_skewness_interval     668
diff_auc_interval          668
dtype: int64


**Ground truth**

In [ ]:
# Load full ground truth (this is done only once)
gt_full_set = pd.read_csv(os.path.join(ground_truth_directory, "individual_ground_truth.csv"))

# Renaming so it has the same names as Pupil
gt_full_set.rename(columns={"Participant": "participant", "Stimulus_Name": "video"}, inplace=True)

# Function to load ground truth file excluding one participant for Leave One Participant Out
def load_ground_truth_exclude(participant_id):
    # Construct the file path
    file_path = os.path.join(
        ground_truth_directory,
        f"Leave-one-out/individual_ground_truth_no_{participant_id}.csv"
    )

    try:
        gt_df = pd.read_csv(file_path)
        return gt_df
    except FileNotFoundError:
        print(f"[Warning] Ground truth file not found for participant {participant_id} at: {file_path}")
        return None 
    
# Function to load ground truth file excluding more participants for Leave Two Participants Out
def load_ground_truth_exclude_multiple(participant_ids):
    assert len(participant_ids) == 2, "Exactly two participant IDs are required."

    id_pairs = [
        (participant_ids[0], participant_ids[1]),
        (participant_ids[1], participant_ids[0])
    ]

    for id1, id2 in id_pairs:
        file_path = os.path.join(
            ground_truth_directory,
            f"Leave-two-out/individual_ground_truth_no_{id1}_no_{id2}.csv"
        )
        if os.path.exists(file_path):
            return pd.read_csv(file_path)
    
    print(f"[Warning] Ground truth file not found for participant pair {participant_ids} in either order.")
    return None 


In [ ]:
def get_train_test_data_fusion(df, gt_full_set, test_pid, target):
    """
    Get the GT data correctly.
    
    For layer-1 fusion:
    - test_pid is the IgnoredParticipant (unseen at layer 0).
    - We merge ground truth based on PredictedParticipant for evaluation.
    """
    # Select test rows where the IgnoredParticipant matches
    test_data = df[df["IgnoredParticipant"] == test_pid].copy()

    # Merge ground truth using PredictedParticipant
    test_gt = gt_full_set[gt_full_set["participant"].isin(test_data["PredictedParticipant"].unique())]
    test_data = test_data.merge(
        test_gt[["participant", "video", target]],
        left_on=["PredictedParticipant", "video"],
        right_on=["participant", "video"],
        how="inner"
    ).drop(columns=["participant"])  # drop duplicate after merge

    # Training data is all rows where the IgnoredParticipant != test_pid
    train_data = df[df["IgnoredParticipant"] != test_pid].copy()
    pid_gt = load_ground_truth_exclude(test_pid) 
    if pid_gt is not None: 
        pid_gt.rename( columns={"Participant": "participant", "Stimulus_Name": "video"}, 
                      inplace=True )
        
    # Merge ground truth for training using PredictedParticipant
    train_data = train_data.merge(
        pid_gt[["participant", "video", target]],
        left_on=["PredictedParticipant", "video"],
        right_on=["participant", "video"],
        how="inner"
    ).drop(columns=["participant"])

    return train_data, test_data



def get_train_test_data_l2po(df, test_pids, target):
    """Split data into train and test and merge with ground truth for L2PO."""
    test_data = df[df["participant"].isin(test_pids)].copy()
    gt = load_ground_truth_exclude(test_pids[0])

    if gt is not None:
        gt.rename(columns={"Participant": "participant", "Stimulus_Name": "video"}, inplace=True)
        test_gt = gt[gt["participant"].isin(test_pids)]
        test_data = test_data.merge(
            test_gt[["participant", "video", target]],
            on=["participant", "video"],
            how="inner",
        )

    train_data = df[~df["participant"].isin(test_pids)].copy()
    
    pid_gt = load_ground_truth_exclude_multiple(test_pids)
    if pid_gt is not None:
        pid_gt.rename(
            columns={"Participant": "participant", "Stimulus_Name": "video"},
            inplace=True
        )
        train_data = train_data.merge(
            pid_gt[["participant", "video", target]],
            on=["participant", "video"],
            how="inner",
        )
        return train_data, test_data
    else:
        return None, None


def evaluate_fusion_model(y_true, y_pred):
    """Calculate evaluation metrics."""
    rmse = root_mean_squared_error(y_true, y_pred)
    nrmse = rmse / (y_true.max() - y_true.min())
    r2 = r2_score(y_true, y_pred)
    corr, p_val = pearsonr(y_true, y_pred)
    return {"NRMSE": nrmse, "R2": r2, "corr": corr, "p": p_val}

def print_filtered_summary(results_df):
    filtered = results_df[
        (results_df['R2'] > -1.0) &                 # Removes models with extremely poor fit
        (results_df['NRMSE'] < 1.5) &              # Removes predictions with very high error
        (results_df['corr'].abs() > 0.2)           # Keeps only modestly correlated results
    ]
    summary_filtered = filtered[['NRMSE', 'R2', 'corr', 'p']].agg(['mean', 'std'])
    print("\n=== Summary (Excluding Outliers) ===")
    print(f"\nFiltered out {len(results_df) - len(filtered)} out of {len(results_df)} participants.")
    print(summary_filtered.round(4))
    excluded = results_df[~results_df.index.isin(filtered.index)]
    print("\nExcluded participants:")
    print(excluded[['participant', 'NRMSE', 'R2', 'corr']])


def print_fusion_weights_summary(fusion_weights):
    if fusion_weights:
        weights_df = pd.DataFrame(fusion_weights).T
        weights_df.columns = [ 'Pupil_weight', 'GSR_weight']
        print("\nAverage Fusion Weights across Participants:")
        print(weights_df.mean().round(4))

In [ ]:
# GSR parameters and features used for the model
gsr_modality_configs = {
    "GSR_arousal": {
        "params": {
            # SVR
            "kernel": "rbf",
            "degree": 2,
            "gamma": 1e-4,
            "coef0": 1,
            "tol": 5e-3,
            "C": 0.5,
            "epsilon": 0.05,

            # LGBM
            "num_leaves": 31,
            "max_depth": -1,
            "learning_rate": 0.1,
            "n_estimators": 300,
            "min_child_samples": 20,
            "subsample": 0.6,
            "colsample_bytree": 0.6,
        },

        "svr_features": [
            'freqKurtEDA_whole', 'freqKurtEDA_interval', 'entropyWavelet_4_interval', 'medianMFCC_1_whole',
            'meanMFCC_1_interval', 'complexity_interval', 'energyDistribution_4_interval',
            'meanDerivative_whole', 'energyDistribution_0_whole', 'energyDistribution_1_whole',
            'energyDistribution_2_whole', 'energyDistribution_3_whole', 'meanNegativeSecondDerivative_whole',
            'phasicPowers_1_whole'
        ],
        "lgbm_features": [
            "freqKurtEDA_whole", "freqKurtEDA_interval", "freqSkewEDA_whole", "freqSkewEDA_interval",
            "energyDistribution_0_whole", "stdMFCC_12_interval", "stdMFCC_11_whole", "stdMFCC_11_interval",
            "stdMFCC_10_interval", "stdMFCC_12_whole", "spectralPowerBand(0.3-0.4)_interval",
            "energyDistribution_1_whole", "energyDistribution_0_interval",
            "spectralPowerBand(0.4-0.5)_interval", "energyWavelet_3_interval", "energyWavelet_2_interval",
            "sumAreas_interval", "meanNegativeSecondDerivative_whole", "energyDistribution_2_whole",
            "energyDistribution_2_interval", "phasicPowers_1_whole", "kurtMFCC_12_interval",
            "entropyWavelet_0_whole", "stdMFCC_10_whole", "energyDistribution_8_interval",
            "energyWavelet_0_interval", "hoc_5_whole", "skewMFCC_12_interval", "skewMFCC_10_interval",
            "energyDistribution_6_whole", "energyDistribution_8_whole", "medianMFCC_10_interval",
            "activity_interval", "meanMFCC_6_interval", "rmsWavelet_3_interval", "energyWavelet_2_whole",
            "hoc_1_whole", "stdMFCC_1_whole", "spectralPowerBand(0.2-0.3)_interval",
            "meanMFCC_11_interval", "medianMFCC_12_interval", "energyDistribution_4_interval",
            "stdMFCC_9_whole", "skewMFCC_1_interval", "skewMFCC_8_interval", "medianMFCC_2_interval",
            "meanSecondDerivative_interval", "meanMFCC_12_interval", "energyDistribution_4_whole",
            "meanMFCC_7_interval"
        ]
    },
        "GSR_valence": {
        "params": {
            # SVR
            "kernel": "rbf",
            "degree": 2,
            "gamma": 5e-4,
            "coef0": 1,
            "tol": 1e-3,
            "C": 1,
            "epsilon": 5e-1,

            # LGBM
            "num_leaves": 31,
            "max_depth": -1,
            "learning_rate": 0.1,
            "n_estimators": 500,
            "min_child_samples": 40,
            "subsample": 0.6,
            "colsample_bytree": 0.6,
        },
        "svr_features": [
            "freqKurtEDA_whole", "freqKurtEDA_interval","kurtMFCC_2_interval",
            "hoc_0_interval", "skewEDA_interval","skewMFCC_5_interval",
            "skewMFCC_6_interval", "meanPeakAmplitude_interval","stdMFCC_2_interval",
            "stdEDA_interval","meanMFCC_3_interval","meanMFCC_11_interval",
            "rmsWavelet_1_interval","meanMFCC_5_interval","mobility_interval","sppw_whole","energyDistribution_0_interval",
        ],
        "lgbm_features": [
            "freqKurtEDA_whole", "freqKurtEDA_interval","freqSkewEDA_interval",
            "freqSkewEDA_whole","skewMFCC_11_interval","energyWavelet_4_interval",
            "rmsWavelet_4_interval","energyDistribution_0_whole","energyWavelet_0_interval",
            "stdMFCC_11_whole", "meanMFCC_12_whole","stdMFCC_12_whole",
            "energyWavelet_2_interval","energyDistribution_2_interval","spectralPowerBand(0.3-0.4)_interval",
            "minSpectralPower_whole","energyDistribution_6_whole","stdMFCC_4_interval",
            "meanMFCC_12_interval","kurtMFCC_2_interval","energyDistribution_8_interval",
            "energyWavelet_3_interval","spectralPowerBand(0.3-0.4)_whole","kurtMFCC_4_interval",
            "stdMFCC_0_interval","energyWavelet_1_interval","meanSecondDerivative_interval",
            "skewMFCC_12_interval","phasicPowers_0_interval","auc_interval",
            "energyDistribution_4_interval","activity_interval","spectralPowerBand(0.4-0.5)_interval",
            "medianMFCC_11_interval","rmsWavelet_3_interval","stdMFCC_12_interval",
            "medianMFCC_10_interval","skewEDA_interval","stdMFCC_10_whole","rmsWavelet_0_interval",
            "kurtMFCC_0_interval", "medianMFCC_12_interval", "stdMFCC_5_interval","energyDistribution_2_whole",
            "stdMFCC_10_interval","varSpectralPower_interval","hoc_2_whole",
            "energyDistribution_1_whole","kurtMFCC_12_whole","meanSecondDerivative_whole",
        ],
    },
}

# Defining the class for the GSR model (stacked SVR LGBM)
class StackedSvrLgbmModel:
    _lgbm_model: LGBMRegressor
    _svr_model: SVR
    _final_regressor: LinearRegression
    svr_features: List[str]
    lgbm_features: List[str]

    def __init__(self,
                 svr_features: List[str],
                 lgbm_features: List[str],
                 # SVR params
                 kernel: str,
                 degree: int,
                 gamma: float,
                 coef0: float,
                 C: float,
                 tol: float,
                 epsilon: float,
                 # LGBM params
                 num_leaves: int,
                 max_depth: int,
                 learning_rate: float,
                 n_estimators: int,
                 min_child_samples: int,
                 subsample: float,
                 colsample_bytree: float):
        self._lgbm_model = LGBMRegressor(n_estimators=n_estimators, num_leaves=num_leaves, max_depth=max_depth,
                                         learning_rate=learning_rate, colsample_bytree=colsample_bytree,
                                         min_child_samples=min_child_samples, subsample=subsample,
                                         n_jobs=-1, boosting_type='dart', force_col_wise=True, verbosity=-1)
        self._svr_model = SVR(kernel=kernel, degree=degree, gamma=gamma, coef0=coef0, tol=tol, C=C, epsilon=epsilon)
        self._final_regressor = LinearRegression()

        self.svr_features = svr_features
        self.lgbm_features = lgbm_features

    def train_and_val(self, train_df: pd.DataFrame, val_df: pd.DataFrame | None,
                      Y_train: pd.DataFrame) -> pd.DataFrame | None:

        X_train_svr = train_df[self.svr_features].to_numpy()
        X_train_lgbm = train_df[self.lgbm_features].astype(float)

        X_train_lgbm, X_val_lgbm, Y_final_regressor_train, Y_final_regressor_val = train_test_split(
            X_train_lgbm,
            Y_train,
            test_size=0.1,
            random_state=42)
        X_train_svr, X_val_svr, _, _ = train_test_split(X_train_svr,
                                                        Y_train,
                                                        test_size=0.1,
                                                        random_state=42)

        self._lgbm_model.fit(X_train_lgbm, Y_final_regressor_train)
        self._svr_model.fit(X_train_svr, Y_final_regressor_train)

        Y_pred_lgbm_train = self._lgbm_model.predict(X_val_lgbm)
        Y_pred_svr_train = self._svr_model.predict(X_val_svr)
        X_final_regressor_train = np.column_stack([Y_pred_lgbm_train, Y_pred_svr_train])
        self._final_regressor.fit(X_final_regressor_train, Y_final_regressor_val)

        if val_df is not None:
            X_test_svr = val_df[self.svr_features].to_numpy()
            X_test_lgbm = val_df[self.lgbm_features].astype(float)

            Y_pred_lgbm_test = self._lgbm_model.predict(X_test_lgbm)
            Y_pred_svr_test = self._svr_model.predict(X_test_svr)
            X_final_regressor_test = np.column_stack([Y_pred_lgbm_test, Y_pred_svr_test])
            Y_pred = self._final_regressor.predict(X_final_regressor_test)
            return Y_pred

        return None

    def test(self, test_df: pd.DataFrame):
        X_test_svr = test_df[self.svr_features].to_numpy()
        X_test_lgbm = test_df[self.lgbm_features].astype(float)

        Y_pred_lgbm_test = self._lgbm_model.predict(X_test_lgbm)
        Y_pred_svr_test = self._svr_model.predict(X_test_svr)

        X_final_regressor_test = np.column_stack([Y_pred_lgbm_test, Y_pred_svr_test])
        Y_pred = self._final_regressor.predict(X_final_regressor_test)

        return Y_pred


## Arousal

In [21]:
# Loading the pupil arousal parameters
pupil_arousal_params = pd.read_csv(os.path.join(home_directory, "pupil_params_arousal.csv"))

In [ ]:
# List to add all fusion inputs
fusion_inputs = []

participants = df["participant"].unique()

for first_p in participants:
    for second_p in participants:
        if first_p == second_p:
            continue
        
        # One participant is ignored (first_p) and one participant is predicted (second_p)
        print(f"Leave out: {first_p}, {second_p}")
        test_id = second_p

        train_data, test_data = get_train_test_data_l2po(
            df, [first_p, second_p], "Arousal"
        )
        if train_data is None or test_data is None:
            continue

        y_train = train_data["Arousal"].values
        y_test = test_data[test_data["participant"] == test_id]["Arousal"].values
        if len(y_test) == 0:
            continue

        # ---- Pupil model-----
        X_train_pupil = train_data[pupil_features]
        X_test_pupil = test_data[pupil_features]
        
        # Making sure Participant column is treated as string
        pupil_arousal_params['Participant'] = pupil_arousal_params['Participant'].astype(str)

        # Convert test_pid to string for matching
        test_pid_str = str(test_id)

        # Find the participant's best params row
        participant_params_row = pupil_arousal_params[pupil_arousal_params['Participant'] == test_pid_str]

        if participant_params_row.empty:
            print(f"No parameters found for participant {test_pid_str}. Using default.")
            model_pupil = LGBMRegressor(random_state=42)
        else:
            # Drop 'Participant' and convert remaining row to dict
            param_dict = participant_params_row.drop(columns=['Participant']).iloc[0].to_dict()

            # Convert any float to int if needed
            param_dict = {k: int(v) if isinstance(v, float) and v.is_integer() else v for k, v in param_dict.items()}

            # Add random_state explicitly
            param_dict['random_state'] = 42

            # Initialize model with participant's parameters
            model_pupil = LGBMRegressor(**param_dict)
        
        model_pupil.fit(X_train_pupil, y_train)
        test_pupil_preds = model_pupil.predict(X_test_pupil)

        # ----- GSR model -----
        X_train_gsr = train_data[gsr_features]
        X_test_gsr = test_data[gsr_features]
        model_gsr = StackedSvrLgbmModel(
            svr_features=gsr_modality_configs["GSR_arousal"]["svr_features"],
            lgbm_features=gsr_modality_configs["GSR_arousal"]["lgbm_features"],
            **gsr_modality_configs["GSR_arousal"]["params"]
        )
        model_gsr.train_and_val(X_train_gsr, None, y_train)
        test_gsr_preds = model_gsr.test(X_test_gsr)

        # Store the fusion inputs per row in test set (even if more than one row)
        # Get the rows corresponding to the current test participant
        test_rows = test_data[test_data["participant"] == test_id].reset_index(drop=True)
        
        for i in range(len(y_test)):
            fusion_inputs.append({
                "IgnoredParticipant": first_p,
                "PredictedParticipant": test_id,
                "video": test_rows.loc[i, "video"],
                "pupil_pred": test_pupil_preds[i],
                "gsr_pred": test_gsr_preds[i],
            })

# Convert to DataFrame
fusion_df = pd.DataFrame(fusion_inputs)

print(fusion_df)

Leave out: 22DFx, 4FoNM
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '4FoNM'] in either order.
Leave out: 22DFx, 5BJD3
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '5BJD3'] in either order.
Leave out: 22DFx, 5KB3V
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '5KB3V'] in either order.
Leave out: 22DFx, 6GSd4
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni o

In [ ]:
def run_participant_loop_arousal(df, gt_full_set, fusion_model_cls, fusion_model_kwargs={}):
    """  Runs Leave-Two-Participant-Out (L2PO) cross-validation for arousal prediction using fusion models built on previously generated modality predictions. """
    participants = df["IgnoredParticipant"].unique()
    participant_results = []
    
    fusion_weights = {}
    all_fusion_preds, all_true_values, all_test_participants = [], [], []
    rmse_per_video = []
    
    true_arousal_max = gt_full_set['Arousal'].max()
    true_arousal_min = gt_full_set['Arousal'].min()
  
    for test_pid in participants:
        train_data, test_data = get_train_test_data_fusion(df, gt_full_set, test_pid, "Arousal")
        if train_data is None or test_data is None:
            continue

        # Extract train/test labels
        y_train = train_data["Arousal"].values
        y_test = test_data["Arousal"].values

        # Get features
        fusion_X_train = train_data[["pupil_pred", "gsr_pred"]]
        fusion_X_test = test_data[["pupil_pred", "gsr_pred"]]

        # Normalize
        fusion_scaler = StandardScaler()
        fusion_X_train = fusion_scaler.fit_transform(fusion_X_train)
        fusion_X_test = fusion_scaler.transform(fusion_X_test)

        # Train fusion model
        model_fusion = fusion_model_cls(**fusion_model_kwargs)
        model_fusion.fit(fusion_X_train, y_train)
        fusion_preds = model_fusion.predict(fusion_X_test)

        # Save fusion weights if linear model used
        if hasattr(model_fusion, "coef_"):
            fusion_weights[test_pid] = model_fusion.coef_

        # Evaluate fusion model
        fusion_metrics = evaluate_fusion_model(y_test, fusion_preds)
        fusion_metrics["participant"] = test_pid

        all_metrics = {
            "participant": test_pid,
            **fusion_metrics,

        }
        
        # Adding the RMSE per video
        for video_id in test_data["video"].unique():
            mask = test_data["video"] == video_id
            y_true_video = y_test[mask]
            y_pred_video = fusion_preds[mask]
        
            if len(y_true_video) == 0 or len(y_pred_video) == 0:
                continue
        
            rmse = np.sqrt(mean_squared_error(y_true_video, y_pred_video))
        
            rmse_per_video.append({
                "Participant": test_pid,
                "Video": video_id,
                "Modality": "Fusion",
                "RMSE": rmse
            })
        participant_results.append(all_metrics)
        all_fusion_preds.extend(fusion_preds)
        all_true_values.extend(y_test)
        all_test_participants.extend([test_pid] * len(y_test))
        
    nrmse_df = pd.DataFrame(rmse_per_video)
    nrmse_df["NRMSE"] = nrmse_df["RMSE"] / (true_arousal_max - true_arousal_min)
    nrmse_df.to_csv("D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/nrmse_per_video_fusion_arousal.csv", index=False)

    return (
        participant_results,
        fusion_weights,
        all_fusion_preds,
        all_true_values,
        all_test_participants,
    )

**Linear Regression**

In [ ]:
print("Linear Regression --------------- Arousal")

fusion_model_cls = LinearRegression
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_arousal(fusion_df,gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Linear Regression --------------- Arousal

=== Summary (Excluding Outliers) ===

Filtered out 1 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0634  0.9162  0.9574  0.0
std   0.0040  0.0118  0.0061  0.0

Excluded participants:
  participant     NRMSE        R2      corr
5        Bs73  0.500908 -2.996558 -0.942155

Average Fusion Weights across Participants:
Pupil_weight    0.0328
GSR_weight      0.1497
dtype: float64


**Ridge Regression**

In [ ]:
print("Ridge --------------- Arousal")
fusion_model_cls = Ridge
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_arousal(fusion_df, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Ridge --------------- Arousal

=== Summary (Excluding Outliers) ===

Filtered out 1 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0634  0.9162  0.9574  0.0
std   0.0040  0.0118  0.0061  0.0

Excluded participants:
  participant     NRMSE        R2      corr
5        Bs73  0.500906 -2.996528 -0.942155

Average Fusion Weights across Participants:
Pupil_weight    0.0328
GSR_weight      0.1497
dtype: float64


**Random Forest Regressor**

In [ ]:
print("RF Regressor -------------- Arousal")
fusion_model_cls = RandomForestRegressor
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_arousal(fusion_df,gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

RF Regressor -------------- Arousal

=== Summary (Excluding Outliers) ===

Filtered out 1 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0598  0.9255  0.9622  0.0
std   0.0039  0.0106  0.0053  0.0

Excluded participants:
  participant     NRMSE        R2      corr
5        Bs73  0.499379 -2.972188 -0.945283


**Gradient Boosting Regressor**

In [ ]:
print("Gradient Boosting Regressor -------------- Arousal")
fusion_model_cls = GradientBoostingRegressor
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_arousal(fusion_df,gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Gradient Boosting Regressor -------------- Arousal

=== Summary (Excluding Outliers) ===

Filtered out 1 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0571  0.9322  0.9656  0.0
std   0.0037  0.0099  0.0051  0.0

Excluded participants:
  participant     NRMSE        R2      corr
5        Bs73  0.496877 -2.932495 -0.951209


**SVR**

In [ ]:
print("SVR -------------- Arousal")
fusion_model_cls = SVR
fusion_model_kwargs = {'kernel':'rbf', 'C':1.0, 'epsilon':0.1}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_arousal(fusion_df,gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

SVR -------------- Arousal

=== Summary (Excluding Outliers) ===

Filtered out 1 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0643  0.9141  0.9604  0.0
std   0.0031  0.0088  0.0043  0.0

Excluded participants:
  participant     NRMSE       R2     corr
5        Bs73  0.512982 -3.19154 -0.95007


## Valence

In [40]:
# Loading the pupil arousal parameters
pupil_valence_params = pd.read_csv(os.path.join(home_directory, "pupil_params_valence.csv"))

In [ ]:
# List to collect all fusion inputs
fusion_inputs = []

participants = df["participant"].unique()

for first_p in participants:
    for second_p in participants:
        if first_p == second_p:
            continue
        
        # One participant is ignored (first_p) and one participant is predicted (second_p)
        print(f"Leave out: {first_p}, {second_p}")
        test_id = second_p

        train_data, test_data = get_train_test_data_l2po(
            df, [first_p, second_p], "Valence"
        )
        if train_data is None or test_data is None:
            continue

        y_train = train_data["Valence"].values
        y_test = test_data[test_data["participant"] == test_id]["Valence"].values
        if len(y_test) == 0:
            continue

        # ----- Pupil model -----
        X_train_pupil = train_data[pupil_features]
        X_test_pupil = test_data[pupil_features]
        
        # Make sure Participant column is treated as string
        pupil_valence_params['Participant'] = pupil_valence_params['Participant'].astype(str)

        # Convert test_pid to string for matching
        test_pid_str = str(test_id)

        # Find the participant's best params row
        participant_params_row = pupil_valence_params[pupil_valence_params['Participant'] == test_pid_str]

        if participant_params_row.empty:
            print(f"No parameters found for participant {test_pid_str}. Using default.")
            model_pupil = LGBMRegressor(objective='quantile', alpha=0.5, random_state=42)
        else:
            # Drop 'Participant' and convert remaining row to dict
            param_dict = participant_params_row.drop(columns=['Participant']).iloc[0].to_dict()

            # Convert any float to int if needed
            param_dict = {k: int(v) if isinstance(v, float) and v.is_integer() else v for k, v in param_dict.items()}

            # Add random_state and other parameters explicitly
            param_dict['random_state'] = 42
            param_dict['objective'] = 'quantile'
            param_dict['alpha'] = 0.5

            # Initialize model with participant's parameters
            model_pupil = LGBMRegressor(**param_dict)
        
        
        model_pupil.fit(X_train_pupil, y_train)
        test_pupil_preds = model_pupil.predict(X_test_pupil)

        # ----- GSR model ------
        X_train_gsr = train_data[gsr_features]
        X_test_gsr = test_data[gsr_features]
        model_gsr = StackedSvrLgbmModel(
            svr_features=gsr_modality_configs["GSR_valence"]["svr_features"],
            lgbm_features=gsr_modality_configs["GSR_valence"]["lgbm_features"],
            **gsr_modality_configs["GSR_valence"]["params"]
        )
        model_gsr.train_and_val(X_train_gsr, None, y_train)
        test_gsr_preds = model_gsr.test(X_test_gsr)

        # Store fusion inputs per row in test set (even if more than one row)
        # Get the rows corresponding to the current test participant
        test_rows = test_data[test_data["participant"] == test_id].reset_index(drop=True)
        
        for i in range(len(y_test)):
            fusion_inputs.append({
                "IgnoredParticipant": first_p,
                "PredictedParticipant": test_id,
                "video": test_rows.loc[i, "video"],
                "pupil_pred": test_pupil_preds[i],
                "gsr_pred": test_gsr_preds[i],
            })

# Convert to DataFrame
fusion_df_valence = pd.DataFrame(fusion_inputs)

print(fusion_df_valence)

Leave out: 22DFx, 4FoNM
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '4FoNM'] in either order.
Leave out: 22DFx, 5BJD3
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '5BJD3'] in either order.
Leave out: 22DFx, 5KB3V
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth\Leave-one-out/individual_ground_truth_no_22DFx.csv
[Warning] Ground truth file not found for participant pair ['22DFx', '5KB3V'] in either order.
Leave out: 22DFx, 6GSd4
[Warning] Ground truth file not found for participant 22DFx at: D:/MASTER/Uni o

In [ ]:
def run_participant_loop_valence(df, gt_full_set, fusion_model_cls, fusion_model_kwargs={}):
    """  Runs Leave-Two-Participant-Out (L2PO) cross-validation for arousal prediction using fusion models built on previously generated modality predictions."""
    participants = df["IgnoredParticipant"].unique()
    participant_results = []
    fusion_weights = {}
    all_fusion_preds, all_true_values, all_test_participants = [], [], []
    rmse_per_video = []
    true_valence_max = gt_full_set['Valence'].max()
    true_valence_min = gt_full_set['Valence'].min()
    for test_pid in participants:
        train_data, test_data = get_train_test_data_fusion(df, gt_full_set, test_pid, "Valence")
        if train_data is None or test_data is None:
            continue

        # Extract train/test labels
        y_train = train_data["Valence"].values
        y_test = test_data["Valence"].values

        # Get features
        fusion_X_train = train_data[["pupil_pred", "gsr_pred"]]
        fusion_X_test = test_data[["pupil_pred", "gsr_pred"]]

        # Normalize
        fusion_scaler = StandardScaler()
        fusion_X_train = fusion_scaler.fit_transform(fusion_X_train)
        fusion_X_test = fusion_scaler.transform(fusion_X_test)

        # Train fusion model
        model_fusion = fusion_model_cls(**fusion_model_kwargs)
        model_fusion.fit(fusion_X_train, y_train)
        fusion_preds = model_fusion.predict(fusion_X_test)

        # Save fusion weights if linear model used
        if hasattr(model_fusion, "coef_"):
            fusion_weights[test_pid] = model_fusion.coef_

        # Evaluate fusion model
        fusion_metrics = evaluate_fusion_model(y_test, fusion_preds)
        fusion_metrics["participant"] = test_pid

        all_metrics = {
            "participant": test_pid,
            **fusion_metrics,

        }
        
        # Add RMSE per video
        for video_id in test_data["video"].unique():
            mask = test_data["video"] == video_id
            y_true_video = y_test[mask]
            y_pred_video = fusion_preds[mask]
        
            if len(y_true_video) == 0 or len(y_pred_video) == 0:
                continue
        
            rmse = np.sqrt(mean_squared_error(y_true_video, y_pred_video))
        
            rmse_per_video.append({
                "Participant": test_pid,
                "Video": video_id,
                "Modality": "Fusion",
                "RMSE": rmse
            })
            
        participant_results.append(all_metrics)
        all_fusion_preds.extend(fusion_preds)
        all_true_values.extend(y_test)
        all_test_participants.extend([test_pid] * len(y_test))
        
    nrmse_df = pd.DataFrame(rmse_per_video)
    nrmse_df["NRMSE"] = nrmse_df["RMSE"] / (true_valence_max - true_valence_min)
    nrmse_df.to_csv("D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/nrmse_per_video_fusion_valence.csv", index=False)
    
    return (
        participant_results,
        fusion_weights,
        all_fusion_preds,
        all_true_values,
        all_test_participants,
    )


**Linear Regression**

In [ ]:
print("Linear Regression --------------- Valence")

fusion_model_cls = LinearRegression
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_valence(fusion_df_valence, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Linear Regression --------------- Valence

=== Summary (Excluding Outliers) ===

Filtered out 0 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0855  0.8636  0.9295  0.0
std   0.0016  0.0047  0.0025  0.0

Excluded participants:
Empty DataFrame
Columns: [participant, NRMSE, R2, corr]
Index: []

Average Fusion Weights across Participants:
Pupil_weight    0.0604
GSR_weight      0.2744
dtype: float64


**Ridge Valence**

In [ ]:
print("Ridge --------------- Valence")
fusion_model_cls = Ridge
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_valence(fusion_df_valence, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Ridge --------------- Valence

=== Summary (Excluding Outliers) ===

Filtered out 0 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0855  0.8636  0.9295  0.0
std   0.0016  0.0047  0.0025  0.0

Excluded participants:
Empty DataFrame
Columns: [participant, NRMSE, R2, corr]
Index: []

Average Fusion Weights across Participants:
Pupil_weight    0.0604
GSR_weight      0.2744
dtype: float64


**Random Forest Regressor**

In [ ]:
print("RF Regressor -------------- Valence")
fusion_model_cls = RandomForestRegressor
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_valence(fusion_df_valence, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

RF Regressor -------------- Valence

=== Summary (Excluding Outliers) ===

Filtered out 0 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0813  0.8764  0.9364  0.0
std   0.0023  0.0069  0.0036  0.0

Excluded participants:
Empty DataFrame
Columns: [participant, NRMSE, R2, corr]
Index: []


**Gradient Boosting Regressor**

In [ ]:
print("Gradient Boosting Regressor -------------- Valence")
fusion_model_cls = GradientBoostingRegressor
fusion_model_kwargs = {}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_valence(fusion_df_valence, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

Gradient Boosting Regressor -------------- Valence

=== Summary (Excluding Outliers) ===

Filtered out 0 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0768  0.8897  0.9433  0.0
std   0.0018  0.0048  0.0025  0.0

Excluded participants:
Empty DataFrame
Columns: [participant, NRMSE, R2, corr]
Index: []


**SVR**

In [ ]:
print("SVR -------------- Valence")
fusion_model_cls = SVR
fusion_model_kwargs = {'kernel':'rbf', 'C':1.0, 'epsilon':0.1}

participant_results, fusion_weights, all_fusion_preds, all_true_values, all_test_participants = run_participant_loop_valence(fusion_df_valence, gt_full_set, fusion_model_cls, fusion_model_kwargs)

results_df = pd.DataFrame(participant_results)
print_filtered_summary(results_df)
print_fusion_weights_summary(fusion_weights)

SVR -------------- Valence

=== Summary (Excluding Outliers) ===

Filtered out 0 out of 51 participants.
       NRMSE      R2    corr    p
mean  0.0780  0.8863  0.9416  0.0
std   0.0016  0.0045  0.0024  0.0

Excluded participants:
Empty DataFrame
Columns: [participant, NRMSE, R2, corr]
Index: []
